**Función:** crea sub dataset 10k (limpio)

**Requiere:**
- ../datasets/df_10k.json

**Crea:**
- ../datasets/df_10k.json

**ATENCIÓN:** en esta versión el fichero de salida tienen el mismo nombre que el fichero de entrada. Se recomienda crear una cópia del fichero 'datasets/df_10k.json' antes de ejecutar este notebook.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
ejecutadoEnGoogleDrive = False # True si se ejecuta en Google Drive

In [ ]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes'
else:
    PATH = '../'

In [3]:
df = pd.read_json(PATH + "datasets/df_10k.json")
df.head()

,photo_id,business_id,caption,label
0,fB-J1F60DgMRoc8eI0z0Ew,JHRlwxxKY0JJcU97rJ-Bug,,food
1,n2hdfzYgxL0twgfSDbro1w,6Y61FisC5lEVxLEaMzyUUQ,,food
2,Z77aLYI_SvAl7PaJ62umJQ,jar_5KMQZtgOseRdUN3L0w,Hazelnut gelato,food
3,tBdw-FR8aNZcJLa5bX-SdA,UB8sxbUN7kPZum22U2O9kg,Mushroom Cheesesteak,food
4,z-ftfswcyAAERjoMzKFJ5A,ArGw19nfPgjR28TTfgb7iw,Oven Roasted Salmon,food


In [ ]:
def load_df(label):
  filename = PATH + 'datasets/{}_duplicates.json'.format(label)
  print('Processing {}'.format(filename))
  df_duplicated = pd.read_json(filename, orient='index')
  df_duplicated['location_clean'] = df_duplicated['location'].apply(lambda x: x.split('/')[-1])
  df_duplicated['duplicates_clean'] = df_duplicated['duplicates'].apply(lambda x: str(x).split('/')[-1][:-2])
  df_duplicated = df_duplicated[['filename', 'duplicates_clean']].reset_index(drop=True)
  return df_duplicated

In [ ]:
duplicated_labels = ['food', 'inside', 'outside', 'drink']
d1 = df.copy().set_index('photo_id')
for label in duplicated_labels:
  d2 = load_df(label)
  d2['photo_id'] = d2.filename.apply(lambda x: x[:-4])
  d2 = d2.set_index('photo_id')
  d1 = d1.join(d2, how='left')
  d1 = d1.loc[d1.duplicates_clean.isna()].drop(columns=['filename','duplicates_clean'])
  print('After processing {} now remain {} photos'.format(label, len(d1)))
 

Processing /content/drive/Shareddrives/The Valley - TFM/DFs   Json/log_duplicados/food_duplicates.json
After processing food now remain 9596 photos
Processing /content/drive/Shareddrives/The Valley - TFM/DFs   Json/log_duplicados/inside_duplicates.json
After processing inside now remain 9556 photos
Processing /content/drive/Shareddrives/The Valley - TFM/DFs   Json/log_duplicados/outside_duplicates.json
After processing outside now remain 9548 photos
Processing /content/drive/Shareddrives/The Valley - TFM/DFs   Json/log_duplicados/drink_duplicates.json
After processing drink now remain 9454 photos


In [ ]:
d1 = d1.reset_index()
d1.head()

,photo_id,business_id,caption,label
0,fB-J1F60DgMRoc8eI0z0Ew,JHRlwxxKY0JJcU97rJ-Bug,,food
1,n2hdfzYgxL0twgfSDbro1w,6Y61FisC5lEVxLEaMzyUUQ,,food
2,Z77aLYI_SvAl7PaJ62umJQ,jar_5KMQZtgOseRdUN3L0w,Hazelnut gelato,food
3,tBdw-FR8aNZcJLa5bX-SdA,UB8sxbUN7kPZum22U2O9kg,Mushroom Cheesesteak,food
4,z-ftfswcyAAERjoMzKFJ5A,ArGw19nfPgjR28TTfgb7iw,Oven Roasted Salmon,food


In [ ]:
json_ = d1.to_json(PATH + '/datasets/df_10k.json', orient = 'records')